In [ ]:
import pickle
import warnings
import scipy.stats
import numpy as np
import panel as pn
import pandas as pd
import caiman as cm
import panel as pn
import holoviews as hv
import events_astro as evs
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from holoviews.streams import Stream, param
from scipy.ndimage.measurements import center_of_mass
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=FutureWarning)
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    warnings.filterwarnings("ignore",category=FutureWarning)
    import utilsss as ut
hv.extension('bokeh')


# Boxes and boxes average traces
Widgets to scroll Scores for the detection and box id 

In [ ]:
#data_path = '/home/pedro/Work/Hippocampus/1photon/downsampled'
#data_path = '/media/pedro/DATAPART1/Zebastiano/DualColor/2019Dec10/5098/TSeries-12082019-1046-2512/Analysis'
#data_path = '/media/pedro/DATAPART1/AGOnIA/datasets_figure/prueba'
#data_path = '/media/pedro/Elements/Zebastiano/VR_2P/PAPER_ANALYSIS/VR_Respawn_analysis_Neurons/prueba'
#data_path = '/media/pedro/DATAPART1/Sara/2P_VR_Data/segmentation_test/room1'
data_path = '/media/pedro/DATAPART1/Sara/2P_VR_Data/ROOM_4/2020Feb11/0003/motion_corrected/TSeries-02112020-1107-1335'

ut.boxes_exploration(data_path)

# Add the seeded-caiman traces and make the plot interactive
Click into a cell to select the cell to display. Widget of Score still present. 
When clicking into a non box location the plots becomes blank and have to change the score to reset. #TODO deal with that error

In [ ]:
#data_path = '/home/pedro/Work/Hippocampus/1photon/downsampled'
#data_path = '/media/pedro/DATAPART1/AGOnIA/datasets_figure/prueba'
#data_path = '/media/pedro/Elements/Zebastiano/VR_2P/PAPER_ANALYSIS/VR_Respawn_analysis_Neurons/prueba'
data_path = '/media/pedro/DATAPART1/Sara/2P_VR_Data/ROOM_4/2020Feb10/0001/motion_corrected/TSeries-02102020-1413-1324'
ut.boxes_exploration_interactive(data_path)

In [ ]:
data_path = '/media/pedro/DATAPART1/Sara/2P_VR_Data/ROOM_4/2020Feb10/0001/motion_corrected/TSeries-02102020-1413-1325'

ut.boxes_exploration_interactive(data_path)

## Using panel to change the location of the widget

In [ ]:
mapa_total = ut.boxes_exploration_interactive(data_path)
pn.panel(mapa_total,widget_location='top_left')

## A way of sorting the plots how we want that doesnt work
Apparently there's no way to do this without fucking up everything. here https://github.com/holoviz/holoviews/issues/91 say's basically that, there's no way of making the layouts more flexible. 

In [ ]:
gspec = pn.GridSpec(sizing_mode='stretch_both', max_height=800)
gspec[2:10,0] = mapa_total[0]
gspec[2:6,2] = mapa_total[1]
gspec[6:10,2] = mapa_total[2]
pn.panel(gspec,widget_location='right')

# Load DLC data and see the occupation

In [ ]:
filename = '/media/pedro/DATAPART1/Zebastiano/Inscopix/2019Mar29/3595/Basler acA800-510um (22829209)_20190329_170429783DLC_resnet50_mouse_trackingOct14shuffle1_50000.h5'
cnm = cnmf.load_CNMF('/home/pedro/Work/Hippocampus/1photon/downsampled/mc_preprocessed_spacial_downsampled2_20190329170534_0_analysis_results_Caiman_refit.hdf5')
cnm.estimates.detrend_df_f(quantileMin=50,frames_window=2000,detrend_only=False)
#result = pd.read_hdf(filename)
#scorer = result.columns.get_level_values(0)[0]
result = ut.read_dlc_results(filename)
result

## Downsample bodyposition a-lo-cabeza
Average neighbouring positions in time. This is done to correlate with calcium activity

In [ ]:
bodypart = 'leftear'
X =np.array([(result[bodypart]['x'][index+1]+result[bodypart]['x'][index])/2 for index in np.linspace(0,
                        result[bodypart].index[-2],result[bodypart].index[-2]/2).astype(int)])
Y =np.array([198-(result[bodypart]['y'][index+1]+result[bodypart]['y'][index])/2 for index in np.linspace(0,
                        result[bodypart].index[-2],result[bodypart].index[-2]/2).astype(int)])

In [ ]:
cnm.estimates.nb_view_components()

## Interactivly present resuts to see trayectories and activity

In [ ]:
cell = 150
events = evs.eventFinder(cnm.estimates.F_dff[cell])
path = hv.Path([(X,Y,events[0])], vdims='fluorescence')
path.opts(cmap='inferno', color='fluorescence', line_width=2,width=600,height=600)
kdims=[hv.Dimension('t', values=np.linspace(0,len(X)-1,len(X)).astype(int))]
def move_dot(t):
    return hv.Scatter((X[t],Y[t])).opts(color='r', size=4)
dmap = hv.DynamicMap(move_dot, kdims=kdims)


In [ ]:
pn.panel(path*dmap+hv.Curve(cnm.estimates.F_dff[cell],kdims = 'frames',vdims = 'Denoised calcium').opts(width=600)*hv.Curve(events[0]).opts(alpha=.5),widget_location='bottom_right')

In [ ]:
curve_dict = {i: place_activity(i) for i in range(len(cnm.estimates.F_dff))}

kdims = [hv.Dimension('cell', values=np.linspace(0,len(cnm.estimates.F_dff)-1,len(cnm.estimates.F_dff)).astype(int))]
holomap = hv.HoloMap(curve_dict, kdims=kdims).options(framewise=True)
pn.panel(holomap,widget_location='bottom_right')

In [ ]:
def place_activity(cell):
    events = evs.eventFinder(cnm.estimates.F_dff[cell])
    path = hv.Path([(X,Y,events[0])], vdims='fluorescence')
    path.opts(cmap='Greys', color='fluorescence', line_width=2,width=600,height=600)
    return (path+hv.Curve(cnm.estimates.F_dff[cell],kdims = 'frames',vdims = 'Denoised calcium').opts(width=600)*hv.Curve(events[0]).opts(alpha=.5))

kdims=[hv.Dimension('cell', values=np.linspace(0,len(cnm.estimates.F_dff)-1,len(cnm.estimates.F_dff)).astype(int))]
dmap = hv.DynamicMap(place_activity, kdims=kdims)
pn.panel(dmap,widget_location='bottom_right')
